<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/Abnormal_Detect_Time_Serries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, RepeatVector, LeakyReLU
from keras import regularizers
from keras.layers import Flatten, Dense, Dropout, Lambda, TimeDistributed

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import tensorflow as tf
from numpy import random

# df_data = pd.read_csv(
#         r'D:\Private_Documents\111\data.csv', parse_dates=True
#     )


def get_data(timesteps, input_dim, path):
    # read data from file
    # data = np.fromfile(r'D:\Private_Documents\111\data.csv').reshape(419,13)
    df_data = pd.read_csv(
        path, parse_dates=True
    )
    #df_data['value'] = df_data['value'].astype(np.float64)
    #scaler = StandardScaler()
    #df_data['value'] = scaler.fit_transform(df_data['value'].values.reshape(-1, 1))

    data = np.array(df_data['value'])
    L = int(len(data) / input_dim)
    data = data[0:L * input_dim]
    data = data.reshape(-1, input_dim)
    dataX = []

    for i in range(len(data)):
        x = data[i:(i + timesteps), :]
        dataX.append(x)
    return np.array(dataX)


timesteps = 1
input_dim = 30
batch_size = 32
latent_dim = 100
EPOCHS =1000
path_data_train = r'D:\data.csv'
path_data_test = r'D:\test.csv'

data_train = get_data(timesteps, input_dim, path_data_train)
data_test = get_data(timesteps, input_dim, path_data_test)

# data_train = np.expand_dims(data_train, axis=1)
# data_test = np.expand_dims(data_test, axis=1)

print(data_train.shape)
print(data_test.shape)
print(data_test)
minTrain = np.min(data_train)
maxTrain = np.max(data_train)

minTest = np.min(data_test)
maxTest = np.max(data_test)

if minTrain < minTest:
    min = minTrain
else:
    min = minTest
if maxTrain > maxTest:
    max = maxTrain
else:
    max = maxTest



x_test = (data_test - min) / (max - min)
x_train = (data_train - min) / (max - min)

x_train_tensor = tf.convert_to_tensor(x_train, dtype=tf.float32)


def autoencoder_model1(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = LSTM(2*latent_dim, activation='relu', return_sequences=True)(inputs)
    L2 = LSTM(latent_dim, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X.shape[1])(L2)
    L4 = LSTM(latent_dim, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(2*latent_dim, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X.shape[2]))(L5)
    model = Model(inputs=inputs, outputs=output)
    return model



def autoencoder_model(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = LSTM(4 * latent_dim, activation='relu', return_sequences=True)(inputs)
    L2 = LSTM(2*latent_dim, activation='relu', return_sequences=True)(L1)
    L3 = LSTM(latent_dim, activation='relu', return_sequences=False)(L2)
    L4 = RepeatVector(X.shape[1])(L3)
    L5 = LSTM(latent_dim, activation='relu', return_sequences=True)(L4)
    L6 = LSTM(2*latent_dim, activation='relu', return_sequences=True)(L5)
    L7 = LSTM(4*latent_dim, activation='relu', return_sequences=True)(L6)
    output = TimeDistributed(Dense(X.shape[2]))(L7)
    model = Model(inputs=inputs, outputs=output)
    return model


def create_lstm_autoencoder(input_dim, timesteps, latent_dim):
    inputs = Input(shape=(timesteps, input_dim,))
    encoded = LSTM(latent_dim)(inputs)
    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(input_dim, return_sequences=True)(decoded)
    # sequence_autoencoder = Model(inputs, decoded)
    # encoder = Model(inputs, encoded)
    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder


def encoder(encoder_inputs):
    x = LSTM(latent_dim)(encoder_inputs)
    return x

def decoder(x):
    x = RepeatVector(timesteps)(x)
    x = LSTM(input_dim, return_sequences=True)(x)
    return x



#Su dung autoencoder
# optimizer = tf.keras.optimizers.Adam(lr=0.001)
# input = Input(shape=(timesteps, input_dim,))
# endcoder_output = encoder(input)
# decoder_outputs = decoder(endcoder_output)
# model = Model(input, [decoder_outputs])
# model.compile(optimizer="adam", loss="mse")
# model.summary()
# model.fit(x_train, x_train, epochs=10000)

# Su dung create_lstm_autoencoder

model = autoencoder_model1(x_train)
#model = create_lstm_autoencoder(input_dim, timesteps, latent_dim)

model.compile(optimizer="adam", loss="mse")

model.fit(x_train, x_train, batch_size=batch_size,epochs=EPOCHS)

x_train_pred = model.predict(x_train, batch_size=batch_size)


threshold =[]
for i in range(0, len(x_train)):
    dist = np.sum(np.square(x_train_pred[i] - x_train[i]))
    #dist = np.linalg.norm(x_train_pred[i] - x_train[i])
    threshold.append(dist)

threshold = np.max(threshold)
print('threshold:',threshold)
x_test_pred = model.predict(x_test, batch_size=batch_size)
threshold_test =[]

for i in range(0, len(x_test)):
    dist = np.sum(np.square(x_test_pred[i] - x_test[i]))
    #dist = np.linalg.norm(x_test_pred[i] - x_test[i])
    threshold_test.append(dist)

anomalies = []

for i in range(0, len(threshold_test)):
    if threshold_test[i] > threshold:
        anomalies.append(1)
    else:
        anomalies.append(0)

print('Number Test Samples:',len(anomalies))
print('Number of anomaly samples:', np.sum(anomalies))

print(anomalies)
print(x_test[0])
print('----------------')
print(x_test_pred[0])
print('---------------')

print('---------------------')
print(x_test[1])
print('----------------')
print(x_test_pred[1])
print('---------------')

print(x_test[2])
print('----------------')
print(x_test_pred[2])
print('---------------')



# for i in range(0, 5):
#
#     t1 = x_train[i].flatten()
#     t2 = x_train_pred[i].flatten()
#     x1 = x_test[i].flatten()
#     y1 = x_test_pred[i].flatten()
#
#     plt.figure(figsize=(18, 10))
#
#     plt.subplot('141')
#     plt.plot(x1, label='test:' + str(i))
#     plt.plot(y1, label='pred test:' + str(i))
#     plt.grid(True)
#     plt.subplot('142')
#     plt.plot(t1, label='train:' + str(i))
#     plt.plot(t2, label='pred train:' + str(i))
#     plt.grid(True)
#
#     plt.show();

